In [4]:
# В этом коде используется датасет вин California wines из sklearn для построения и исследования модели K Nearest Neighbors.
# Сначала данные загружаются в Pandas DataFrame для удобства работы. Изучается описание, проверяется отсутствие пропущенных значений, распределение классов.
# Далее данные разбиваются на обучающую и тестовую выборки. Строится парный график зависимостей признаков (pairplot).
# Проверяется необходимость масштабирования данных, после чего признаки масштабируются при помощи MinMaxScaler.
# Затем запускается цикл перебора количества соседей для модели KNN от 2 до 10. Для каждого варианта строится модель, оценивается качество на тестовой выборке. Лучшее значение запоминается.
# В конце строится матрица ошибок для лучшей модели и выводится лучшее количество соседей.
# Таким образом, в работе подбираются гиперпараметры и исследуется качество модели KNN на датасете вин.

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import scikitplot as skplt
import missingno as msno

wine = load_wine()

ModuleNotFoundError: No module named 'scikitplot'

In [ ]:

print(wine.DESCR)

In [ ]:
# В Pandas я создаю фрейм данных для более упорядоченного отображения данных и проверяю соответствие предоставленному описанию:
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['label'] = wine.target
df.head()

In [ ]:
#проверяем процентное представительство классов
df.label.value_counts(normalize=True).round(3)

In [ ]:
# Проверка на null
df.info()

In [ ]:
# Копирую нужные мне столбцы
y = df["label"].copy()
X = df.iloc[:, :-1].copy()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=0)


In [ ]:
# для визуального исследования данных
sns.pairplot(df,
             hue='label',
             palette="tab10",
             corner=True)

plt.show()

In [ ]:
# Проверка нужна ли масштабируемости
df.describe().loc[['min', 'max']].T

In [ ]:
# Стабилизирую данные
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Указываю соседей
best = 0
c = 0
for i in range(2, 11):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    n = np.mean(y_pred == y_test)
    if n > best:
        best = n
        c = i
print(best, c)

In [ ]:
# Этот код строит матрицу неточностей или матрицу ошибок (confusion matrix) для результатов классификации.
knn = KNeighborsClassifier(n_neighbors=c)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=False)
plt.show()